In [ ]:
import iql
%load_ext iql

#### Cell and Line Magics

You can use %iql for a one-line command, or %%iql at the beginning of a cell for a long query.

This is the same as calling: `iql.execute(...)`

In [ ]:
# %iql is called a "line magic" in Jupyter. %%iql is a cell magic

df = %iql select * from range(10)

display(df)

#### Multiple Statements Separated by Semicolons

In [ ]:
%%iql

create table abc as select * from range(10);

select * from abc where range > 5

#### BQL can be executed on the fly

Use -o to store the output in a variable

In [ ]:
%%iql -o my_bql_df

get(px_last)
for('IBM US Equity')
with(dates=range(-3d, 0d))

#### %%capture to Suppress Display

In [ ]:
%%capture 
%%iql -o my_bql_df

get(px_last)
for('IBM US Equity')
with(dates=range(-3d, 0d))

#### Now for interesting stuff: SQL + BQL

Nothing too fancy, just ordering the results

In [ ]:
%%iql -o my_bql_df

select * from bql("
    get(px_last)
    for('IBM US Equity')
    with(dates=range(-3d, 0d))
")
order by date desc

#### Pivoting multiple fields for a time series result



In [ ]:
import iql
%load_ext iql

#### Cell and Line Magics

You can use %iql for a one-line command, or %%iql at the beginning of a cell for a long query.

This is the same as calling: `iql.execute(...)`

In [ ]:
# %iql is called a "line magic" in Jupyter. %%iql is a cell magic

df = %iql select * from range(10)

display(df)

#### Multiple Statements Separated by Semicolons

In [ ]:
%%iql

create table abc as select * from range(10);

select * from abc where range > 5

#### BQL can be executed on the fly

Use -o to store the output in a variable

In [ ]:
%%iql -o my_bql_df

get(px_last)
for('IBM US Equity')
with(dates=range(-3d, 0d))

#### Pivot Multi-Field Data

In [ ]:
%%iql -o my_bql_df

select * from bql("
    get(px_last, px_volume)
    for('IBM US Equity')
    with(dates=range(-3d, 0d))
", pivot=((date, id), name, value))
order by date desc

#### Or, pivot to one column per security

In [ ]:
%%iql -o my_bql_df

--- pivot=(rows, columns, value field)
select * from bql("
    get(px_last)
    for(members('NIFTY Index'))
    with(dates=range(-3d, 0d))
", pivot=(date, id, value))


#### Other Examples

In [ ]:
%%iql

---- Ranking

SELECT *, rank() over (partition by month(date) order by px_high) ranking
    FROM bql("
                get (px_open, px_high, px_low, px_last, px_volume/1M as #px_volume)
                for ('IBM US Equity')
                with (dates=range(-1Y, 0D), fill=prev)
            ", pivot=(auto)) as data
    ORDER BY date

In [ ]:
%%iql

---- Skew and Kurtosis

SELECT id,
    avg(px_last) avg_px_last,
    kurtosis(px_last) kurt_px_last,
    count(*) num_observations
    FROM bql("
            get(px_last(dates=range(-10D, 0D), fill=prev) as #px_last)
            for (['TSLA US Equity','IBM US Equity'])
            ", pivot=auto)
    GROUP BY id

In [ ]:
%%iql 

---- Price change vs peers

SELECT c.id, c.country_iso, c.industry_subgroup, c.cur_mkt_cap, c.px_change, p.id, p.country_iso, p.industry_subgroup, p.cur_mkt_cap, p.px_change, c.px_change - p.px_change px_change_delta
    FROM
    (SELECT *, (px_last - px_last_1y) / px_last_1y as px_change
        -- px_change could also be calculated in BQL via pct_diff()
        FROM
            bql("
                get(name, COUNTRY_ISO, industry_subgroup, cur_mkt_cap/1M as #cur_mkt_cap, px_last, px_last(dates=-1Y) as #px_last_1y) for (peers('IBM US Equity')) with (currency=USD)
            ", pivot=(id,name), dropna=True)
        ) as p
    JOIN
        (SELECT *, (px_last - px_last_1y) / px_last_1y as px_change
        FROM
            bql("
                get(name, COUNTRY_ISO, industry_subgroup, cur_mkt_cap/1M as #cur_mkt_cap, px_last, px_last(dates=-1Y) as #px_last_1y) for ('IBM US Equity') with (currency=USD)
            ", pivot=(id,name))
    ) as c
    -- c always returns one row
    ON true ORDER BY px_change_delta desc 

In [ ]:
%%iql

---- With CTES

WITH c AS (select * from bql("get(px_last) for('IBM US Equity') with(fill=prev, currency=USD)")),
    idx AS (select * from bql("get(px_last) for ('SPX Index') with(fill=prev, currency=USD)"))
    
    SELECT c.id c, c.value as c_px, idx.id idx, idx.value as idx_px FROM c, idx

In [ ]:
%%iql

--- You can create tables in the in-memory database
--- These are deleted. You can also initialize the database with on-disk storage

CREATE TABLE temp_table_1 as 
(SELECT id from bql("get(px_volume) for(translatesymbols(members('SPX Index'), targetidtype=FUNDAMENTALTICKER)) with(currency=USD)", pivot=(id, name)) 
        ORDER BY px_volume desc limit 10) 
    ;
SELECT id, name, px_last, px_volume/1000000 as px_volume FROM bql('get(name, px_last, px_volume) for("$SECURITY")', paramquery=('$SECURITY', 'select * from temp_table_1 limit 2'), pivot=(id, name))
